# Sagemaker Studio Basic Usage

## Print AWS region & setup a sagemaker session

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
import sys

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')

Region = eu-west-2


## Collect the libraries needed and setup details for our S3 bucket

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from time import sleep, gmtime, strftime
import json
import time

In [8]:
rawbucket= sess.default_bucket() # Alternatively you can use our custom bucket here. 

prefix = 'sagemaker-exemplar' # use this prefix to store all files pertaining to this workshop.

dataprefix = prefix + '/data'

## Collect a sample dataset

In [9]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
data = pd.read_excel('default of credit card clients.xls', header=1)
data = data.drop(columns = ['ID'])
data.head()

--2021-01-18 13:26:25--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls.1’

default of credit c 100%[===================>]   5.28M  4.05MB/s    in 1.3s    

2021-01-18 13:26:28 (4.05 MB/s) - ‘default of credit card clients.xls.1’ saved [5539328/5539328]



,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [10]:
data.rename(columns={"default payment next month": "Label"}, inplace=True)
lbl = data.Label
data = pd.concat([lbl, data.drop(columns=['Label'])], axis = 1)
data.head()

,Label,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,1,20000,2,2,1,24,2,2,-1,-1,...,689,0,0,0,0,689,0,0,0,0
1,1,120000,2,2,2,26,-1,2,0,0,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,0,90000,2,2,2,34,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,0,50000,2,2,1,37,0,0,0,0,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,0,50000,1,2,1,57,-1,0,-1,0,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679


In [8]:
if not os.path.exists('rawdata/rawdata.csv'):
    !mkdir rawdata
    data.to_csv('rawdata/rawdata.csv', index=None)
else:
    pass
# Upload the raw dataset
raw_data_location = sess.upload_data('rawdata', bucket=rawbucket, key_prefix=dataprefix)
print(raw_data_location)

s3://sagemaker-eu-west-2-229081625704/sagemaker-exemplar/data


## Data Preprocessing

In [49]:
import os
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.exceptions import DataConversionWarning
from sklearn.compose import make_column_transformer
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

# Create a copy of the raw dataset
df = data.copy()
COLS = df.columns
newcolorder = ['PAY_AMT1','BILL_AMT1'] + list(COLS[1:])[:11] + list(COLS[1:])[12:17] + list(COLS[1:])[18:]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df.drop('Label', axis=1), df['Label'], test_size=0.2, random_state=2)

# concat to ensure Label column is the first column in dataframe
train_full = pd.concat([pd.DataFrame(y_train.values, columns=['Label']), X_train], axis=1)
test_full = pd.concat([pd.DataFrame(y_test.values, columns=['Label']), X_test], axis=1)

# Upload the raw dataset
train_full.to_csv('rawdata/train.csv', index=None)
test_full.to_csv('rawdata/test.csv', index=None)

sess.upload_data('rawdata/train.csv', bucket=rawbucket, key_prefix=dataprefix)
sess.upload_data('rawdata/test.csv', bucket=rawbucket, key_prefix=dataprefix)

's3://sagemaker-eu-west-2-229081625704/sagemaker-exemplar/data/test.csv'

## Train Model

Create an experiment, then a trail in the experiment from which to train a model.

In [15]:
!pip install sagemaker-experiments 
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

  Using cached sagemaker_experiments-0.1.25-py3-none-any.whl (40 kB)


In [16]:
cc_experiment = Experiment.create(
    experiment_name=f"Build-train-deploy-{int(time.time())}", 
    description="Predict credit card default from payments data", 
    sagemaker_boto_client=sm)
print(cc_experiment)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f535f48cb90>,experiment_name='Build-train-deploy-1610976843',description='Predict credit card default from payments data',tags=None,experiment_arn='arn:aws:sagemaker:eu-west-2:229081625704:experiment/build-train-deploy-1610976843',response_metadata={'RequestId': '4a2fde8f-c4db-41bb-87aa-13b8b87ed39d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4a2fde8f-c4db-41bb-87aa-13b8b87ed39d', 'content-type': 'application/x-amz-json-1.1', 'content-length': '101', 'date': 'Mon, 18 Jan 2021 13:34:03 GMT'}, 'RetryAttempts': 0})


In [25]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 'xgboost', '1.0-1')
s3_input_train = sagemaker.s3_input(s3_data='s3://' + rawbucket + '/sagemaker-exemplar/data/train.csv', content_type='csv')

trial_name = f"cc-default-training-job-{int(time.time())}"
cc_trial = Trial.create(
        trial_name=trial_name, experiment_name=cc_experiment.experiment_name, sagemaker_boto_client=sm
    )
cc_training_job_name = "cc-training-job-{}".format(int(time.time()))

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    train_max_run=86400,
                                    output_path='s3://{}/models'.format(rawbucket),
                                    sagemaker_session=sess) # set to true for distributed training

xgb.set_hyperparameters(max_depth=3,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        verbosity=0,
                        objective='binary:logistic',
                        num_round=20)

xgb.fit(inputs = {'train':s3_input_train},
       job_name=cc_training_job_name,
        experiment_config={
            "TrialName": cc_trial.trial_name, #log training job in Trials for lineage
            "TrialComponentDisplayName": "Training",
        },
        wait=True,
    )

INFO:sagemaker:Creating training-job with name: cc-training-job-1610978083


2021-01-18 13:54:43 Starting - Starting the training job...
2021-01-18 13:54:44 Starting - Launching requested ML instances......
2021-01-18 13:55:48 Starting - Preparing the instances for training...
2021-01-18 13:56:38 Downloading - Downloading input data...
2021-01-18 13:57:07 Training - Downloading the training image...
2021-01-18 13:57:43 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[13:57:39] 10873x23 matrix with 245207 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Single 

## Deploy Model to an Endpoint

In [39]:
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker import RealTimePredictor
from sagemaker.predictor import csv_serializer

sm_client = boto3.client('sagemaker')

latest_training_job = sm_client.list_training_jobs(MaxResults=1,
                                                SortBy='CreationTime',
                                                SortOrder='Descending')

training_job_name=TrainingJobName=latest_training_job['TrainingJobSummaries'][0]['TrainingJobName']
training_job_description = sm_client.describe_training_job(TrainingJobName=training_job_name)

model_data = training_job_description['ModelArtifacts']['S3ModelArtifacts']
container_uri = training_job_description['AlgorithmSpecification']['TrainingImage']

In [38]:
def create_model(role, model_name, container_uri, model_data):
    return sm_client.create_model(
        ModelName=model_name,
        PrimaryContainer={
        'Image': container_uri,
        'ModelDataUrl': model_data,
        },
        ExecutionRoleArn=role)
    
try:
    model = create_model(role, training_job_name, container_uri, model_data)
except Exception as e:
    sm_client.delete_model(ModelName=training_job_name)
    model = create_model(role, training_job_name, container_uri, model_data)
        

print('Model created: '+model['ModelArn'])

{'ModelArn': 'arn:aws:sagemaker:eu-west-2:229081625704:model/cc-training-job-1610978083',
 'ResponseMetadata': {'RequestId': 'f5410af2-599b-4f56-86ad-6528e1f1ed0d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f5410af2-599b-4f56-86ad-6528e1f1ed0d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Mon, 18 Jan 2021 14:33:01 GMT'},
  'RetryAttempts': 2}}

This code tells SageMaker to capture 100% of the inference payloads received by the endpoint, capture both inputs and outputs, and also note the input content type as csv.

In [40]:
s3_capture_upload_path = 's3://{}/{}/monitoring/datacapture'.format(rawbucket, 'sagemaker-exemplar')
data_capture_configuration = {
    "EnableCapture": True,
    "InitialSamplingPercentage": 100,
    "DestinationS3Uri": s3_capture_upload_path,
    "CaptureOptions": [
        { "CaptureMode": "Output" },
        { "CaptureMode": "Input" }
    ],
    "CaptureContentTypeHeader": {
       "CsvContentTypes": ["text/csv"],
       "JsonContentTypes": ["application/json"]}}

In [41]:
def create_endpoint_config(model_config, data_capture_config): 
    return sm_client.create_endpoint_config(
                    EndpointConfigName=model_config,
                    ProductionVariants=[
                            {
                                'VariantName': 'AllTraffic',
                                'ModelName': model_config,
                                'InitialInstanceCount': 1,
                                'InstanceType': 'ml.m4.xlarge',
                                'InitialVariantWeight': 1.0,
                    },

                        ],
                    DataCaptureConfig=data_capture_config
                    )


#-----------------------------

try:
    endpoint_config = create_endpoint_config(training_job_name, data_capture_configuration)
except Exception as e:
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint)
    endpoint_config = create_endpoint_config(training_job_name, data_capture_configuration)

print('Endpoint configuration created: '+ endpoint_config['EndpointConfigArn'])

Endpoint configuration created: arn:aws:sagemaker:eu-west-2:229081625704:endpoint-config/cc-training-job-1610978083


create endpoint...

In [42]:
endpoint_name = training_job_name
def create_endpoint(endpoint_name, config_name):
    return sm_client.create_endpoint(
                                    EndpointName=endpoint_name,
                                    EndpointConfigName=training_job_name
                                )


try:
    endpoint = create_endpoint(endpoint_name, endpoint_config)
except Exception as e:
    sm_client.delete_endpoint(EndpointName=endpoint_name)
    endpoint = create_endpoint(endpoint_name, endpoint_config)

print('Endpoint created: '+ endpoint['EndpointArn'])

Endpoint created: arn:aws:sagemaker:eu-west-2:229081625704:endpoint/cc-training-job-1610978083


In [102]:
inf_df = test_full[:5]
inf_df.drop('Label', inplace=True, axis=1)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [73]:
from sagemaker import RealTimePredictor
from sagemaker.predictor import csv_serializer

predictor = RealTimePredictor(endpoint=endpoint_name, content_type = 'text/csv')

# DF row into string of features comma sep.
x = inf_df.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
vals = [','.join(ele.split()) for ele in x]

payload = vals[0].rstrip('\n')
response = predictor.predict(data=payload[2:])

print(response)

b'0.04902255907654762'


In the code, the current_endpoint_capture_prefix captures the directory path where your ModelMonitor outputs are stored. Navigate to your Amazon S3 bucket, to see if the prediction requests are being captured. Note that this location should match the s3_capture_upload_path in the code above.

In [77]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=rawbucket, Prefix='sagemaker-exemplar/monitoring/datacapture/' + endpoint_name + '/AllTraffic')
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))
capture_files[0]

sagemaker-exemplar/monitoring/datacapture/cc-training-job-1610978083/AllTraffic
Found Capture Files:
sagemaker-exemplar/monitoring/datacapture/cc-training-job-1610978083/AllTraffic/2021/01/18/14/58-17-257-22899a66-1b9a-41a6-a682-763804c1b581.jsonl


'sagemaker-exemplar/monitoring/datacapture/cc-training-job-1610978083/AllTraffic/2021/01/18/14/58-17-257-22899a66-1b9a-41a6-a682-763804c1b581.jsonl'

Run the following code to extract the content of one of the json files and view the captured outputs. 

In [94]:
def get_obj_body(bucket, obj_key):
    return s3_client.get_object(Bucket=rawbucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(rawbucket, capture_files[0])
print(capture_file.split('\n')[0])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,-2.0,-2.0,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.04902255907654762","encoding":"CSV"}},"eventMetadata":{"eventId":"aa592c27-51fb-4eea-9968-1943a8a3f487","inferenceTime":"2021-01-18T14:58:17Z"},"eventVersion":"0"}


## Model Monitoring

In this step, you enable SageMaker Model Monitor to monitor the deployed endpoint for data drift. To do so, you compare the payload and outputs sent to the model against a baseline and determine whether there is any drift in the input data, or the label.  

In [123]:
baseline_data_uri = 's3://sagemaker-eu-west-2-229081625704/sagemaker-exemplar/monitoring/' + endpoint_name + '/baselining/data'
baseline_results_uri = 's3://sagemaker-eu-west-2-229081625704/sagemaker-exemplar/monitoring/' + endpoint_name + '/baselining/results'

Run the following code to set up a baseline job for Model Monitor to capture the statistics of the training data. To do this, Model Monitor uses the deequ library built on top of Apache Spark for conducting unit tests on data.  

Model Monitor sets up a separate instance, copies over the training data, and generates some statistics. The service generates a lot of Apache Spark logs, which you can ignore. Once the job is completed, you will see a Spark job completed output.

In [103]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600)

my_default_monitor.suggest_baseline(
    baseline_dataset='s3://' + rawbucket + '/sagemaker-exemplar/data/train.csv',
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)

INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2021-01-18-15-22-40-691



Job Name:  baseline-suggestion-job-2021-01-18-15-22-40-691
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-2-229081625704/sagemaker-exemplar/data/train.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-2-229081625704/sagemaker-exemplar/monitoring/cc-training-job-1610978083/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.......................2021-01-18 15:26:18,315 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:eu-west-2:229081625704:processing-job/baseline-suggestion-job-2021-01-18-15-22-40-691', 'ProcessingJobName': 'baseline-suggestion-job-2021-01-18-15-22-40-691', 'Environment': {'dataset_format': '{"csv": {"header": true, "output_c

In [126]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=rawbucket, Prefix='sagemaker-exemplar/monitoring/cc-training-job-1610978083/baselining/results/')
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
sagemaker-exemplar/monitoring/cc-training-job-1610978083/baselining/results/constraints.json
 sagemaker-exemplar/monitoring/cc-training-job-1610978083/baselining/results/statistics.json


In [127]:
baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,Label,Fractional,24000,4837,0.220875,5.301000e+03,0.414836,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0,..."
1,LIMIT_BAL,Fractional,24000,4837,167735.666667,4.025656e+09,129529.688210,10000.0,800000.0,"[{'lower_bound': 10000.0, 'upper_bound': 89000...",0.64,2048.0,"[[50000.0, 100000.0, 110000.0, 210000.0, 15000..."
2,SEX,Fractional,24000,4837,1.603958,3.849500e+04,0.489073,1.0,2.0,"[{'lower_bound': 1.0, 'upper_bound': 1.1, 'cou...",0.64,2048.0,"[[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,..."
3,EDUCATION,Fractional,24000,4837,1.850292,4.440700e+04,0.791020,0.0,6.0,"[{'lower_bound': 0.0, 'upper_bound': 0.6, 'cou...",0.64,2048.0,"[[2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0,..."
4,MARRIAGE,Fractional,24000,4837,1.551125,3.722700e+04,0.521427,0.0,3.0,"[{'lower_bound': 0.0, 'upper_bound': 0.3, 'cou...",0.64,2048.0,"[[1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0,..."
5,AGE,Fractional,24000,4837,35.466792,8.512030e+05,9.201697,21.0,75.0,"[{'lower_bound': 21.0, 'upper_bound': 26.4, 'c...",0.64,2048.0,"[[34.0, 35.0, 42.0, 38.0, 45.0, 44.0, 48.0, 32..."
6,PAY_0,Fractional,24000,4837,-0.020667,-4.960000e+02,1.121861,-2.0,8.0,"[{'lower_bound': -2.0, 'upper_bound': -1.0, 'c...",0.64,2048.0,"[[0.0, 0.0, 2.0, 0.0, -2.0, 1.0, -2.0, 0.0, 0...."
7,PAY_2,Fractional,24000,4837,-0.139208,-3.341000e+03,1.193418,-2.0,8.0,"[{'lower_bound': -2.0, 'upper_bound': -1.0, 'c...",0.64,2048.0,"[[0.0, 0.0, 2.0, 0.0, -2.0, -1.0, -1.0, 0.0, 0..."
8,PAY_3,Fractional,24000,4837,-0.172542,-4.141000e+03,1.192660,-2.0,8.0,"[{'lower_bound': -2.0, 'upper_bound': -1.0, 'c...",0.64,2048.0,"[[-1.0, 0.0, 2.0, 0.0, -2.0, -1.0, 2.0, 0.0, 0..."
9,PAY_4,Fractional,24000,4837,-0.225500,-5.412000e+03,1.161888,-2.0,8.0,"[{'lower_bound': -2.0, 'upper_bound': -1.0, 'c...",0.64,2048.0,"[[-1.0, 0.0, 2.0, 0.0, -2.0, -1.0, 2.0, 0.0, 0..."


In [128]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,inferred_type,completeness,num_constraints.is_non_negative
0,Label,Fractional,0.832264,True
1,LIMIT_BAL,Fractional,0.832264,True
2,SEX,Fractional,0.832264,True
3,EDUCATION,Fractional,0.832264,True
4,MARRIAGE,Fractional,0.832264,True
5,AGE,Fractional,0.832264,True
6,PAY_0,Fractional,0.832264,False
7,PAY_2,Fractional,0.832264,False
8,PAY_3,Fractional,0.832264,False
9,PAY_4,Fractional,0.832264,False


#### Setup reports

Run the following code to set up the frequency for endpoint monitoring.

You can specify daily or hourly. This code specifies an hourly frequency, but you may want to change this for production applications as hourly frequency will generate a lot of data. Model Monitor will produce a report consisting of all the violations it finds.

In [130]:
reports_prefix = '{}/reports'.format('sagemaker-exemplar/monitoring')
s3_report_path = 's3://{}/{}'.format(rawbucket,reports_prefix)
print(s3_report_path)

s3://sagemaker-eu-west-2-229081625704/sagemaker-exemplar/monitoring/reports


In [132]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_name = 'model-monitor-schedule-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,

)

INFO:sagemaker:Creating monitoring schedule name model-monitor-schedule-2021-01-18-15-41-42.



Creating Monitoring Schedule with name: model-monitor-schedule-2021-01-18-15-41-42


Note that this code enables Amazon CloudWatch Metrics, which instructs Model Monitor to send outputs to CloudWatch. You can use this approach to trigger alarms using CloudWatch Alarms to let engineers or admins know when data drift has been detected.  

#### Generate some test data for model monitoring drift

In [136]:
COLS = data.columns
test_full = pd.read_csv('rawdata/test.csv')
test_full.head()

,Label,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,1.0,20000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,...,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,...,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0
3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
faketestdata = test_full
faketestdata['EDUCATION'] = -faketestdata['EDUCATION'].astype(float)
faketestdata['BILL_AMT2']= (faketestdata['BILL_AMT2']//10).astype(float)
faketestdata['AGE']= (faketestdata['AGE']-10).astype(float)

faketestdata.drop(columns=['Label'], inplace=True)
faketestdata.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000.0,2.0,-2.0,1.0,-6.0,2.0,2.0,-1.0,-1.0,-2.0,...,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90000.0,2.0,-2.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,...,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
# Remove rows containing NAN
faketestdata = faketestdata.dropna()

In [144]:
x = faketestdata.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
vals = [','.join(ele.split()) for ele in x]

for idx in range(0, 50):
    payload = vals[idx].rstrip('\n')
    response = predictor.predict(data=payload[1:])
    print(response)

b'0.10432717949151993'
b'0.07860476523637772'
b'0.0907464474439621'
b'0.0673631951212883'
b'0.07825499773025513'
b'0.10432717949151993'
b'0.08196871727705002'
b'0.11089660972356796'
b'0.07187852263450623'
b'0.10773836076259613'
b'0.10128173977136612'
b'0.11531323194503784'
b'0.10061458498239517'
b'0.10432717949151993'
b'0.08877485990524292'
b'0.10432717949151993'
b'0.11513546854257584'
b'0.07378952205181122'
b'0.0675986036658287'
b'0.07951434701681137'
b'0.07860476523637772'
b'0.10734861344099045'
b'0.09059973806142807'
b'0.07860476523637772'
b'0.069251149892807'
b'0.07337482273578644'
b'0.10363355278968811'
b'0.10432717949151993'
b'0.09654705971479416'
b'0.10214970260858536'
b'0.08859945088624954'
b'0.09259957075119019'
b'0.08503322303295135'
b'0.10214970260858536'
b'0.08495453745126724'
b'0.08585218340158463'
b'0.10432717949151993'
b'0.04899720102548599'
b'0.08244786411523819'
b'0.09921275824308395'
b'0.09356721490621567'
b'0.09018008410930634'
b'0.09506930410861969'
b'0.092357724905

In [145]:
desc_schedule_result = my_default_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))

Schedule status: Scheduled


In [146]:
mon_executions = my_default_monitor.list_executions()
print("We created ahourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(300)
    mon_executions = my_default_monitor.list_executions()

No executions found for schedule. monitoring_schedule_name: model-monitor-schedule-2021-01-18-15-41-42
We created ahourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.
We will have to wait till we hit the hour...
Waiting for the 1st execution to happen...


In [147]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
time.sleep(20)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 5 violations.


In [148]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))
from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=rawbucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

Report Uri: s3://sagemaker-eu-west-2-229081625704/sagemaker-exemplar/monitoring/reports/cc-training-job-1610978083/model-monitor-schedule-2021-01-18-15-41-42/2021/01/18/16
Report bucket: sagemaker-eu-west-2-229081625704
Report key: sagemaker-exemplar/monitoring/reports/cc-training-job-1610978083/model-monitor-schedule-2021-01-18-15-41-42/2021/01/18/16
Found Report Files:
sagemaker-exemplar/monitoring/reports/cc-training-job-1610978083/model-monitor-schedule-2021-01-18-15-41-42/2021/01/18/16/constraint_violations.json
 sagemaker-exemplar/monitoring/reports/cc-training-job-1610978083/model-monitor-schedule-2021-01-18-15-41-42/2021/01/18/16/constraints.json
 sagemaker-exemplar/monitoring/reports/cc-training-job-1610978083/model-monitor-schedule-2021-01-18-15-41-42/2021/01/18/16/statistics.json


## Clean up

In [151]:
my_default_monitor.delete_monitoring_schedule()
time.sleep(10) # actually wait for the deletion

sm.delete_endpoint(EndpointName = endpoint_name)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:eu-west-2:229081625704:endpoint/cc-training-job-1610978083".